In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
file = pd.read_csv('report_with_labels.csv')
keys = file.keys()
print(keys)
file.head(5)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'patient_id', 'study_id', 'report_txt',
       'finding', 'impression', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
       'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
       'Pneumonia', 'Pneumothorax', 'Support Devices'],
      dtype='object')


,Unnamed: 0.1,Unnamed: 0,patient_id,study_id,report_txt,finding,impression,Atelectasis,Cardiomegaly,Consolidation,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,0,0,10000032,50414267,FINAL REPORT\...,"\n There is no focal consolidation, pleural...",\n No acute cardiopulmonary process.\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,1,0,10000032,53189527,FINAL REPORT\...,"\n The cardiac, mediastinal and hilar conto...",\n No acute cardiopulmonary abnormality.\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,2,0,10000032,53911762,FINAL REPORT\...,\n Single frontal view of the chest provide...,\n No acute intrathoracic process.\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,3,0,10000032,56699142,FINAL REPORT\...,\n The lungs are clear of focal consolidati...,\n No acute cardiopulmonary process.\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,4,0,10000764,57375967,FINAL REPORT\...,\n PA and lateral views of the chest provid...,\n Focal consolidation at the left lung bas...,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN


In [3]:
labels = ['Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
       'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
       'Pneumonia', 'Pneumothorax']
detected_txt = ['finding', 'impression']

In [4]:
labels_formated = {labels[i]: 'label_'+labels[i] for i in range(len(labels))} # model recognize labels column with 'label' prefix

In [5]:
datasets = []
for item in detected_txt:

    dataset = file[[item]+labels]

    # dropout whitespaces | na | int 0 | str 0 |'' | ___
    dataset[item]= dataset[item].str.strip()
    dataset = dataset.dropna(subset=[item])
    dataset = dataset[(dataset[item]!=0) & 
                      (dataset[item]!='0') &
                      (dataset[item]!='') &
                      (dataset[item]!='___')]

    dataset[labels] = dataset[labels].fillna(0)
    dataset[labels] = dataset[labels].replace(-1, 2) #uncertain label is 2
    dataset[item]=dataset[item].replace(r'\n','', regex=True)
    dataset[labels]=dataset[labels].astype(int)
    # model recognize labels with 'label' prefix
    dataset = dataset.rename(columns=labels_formated)
    print(dataset.info())

    outdir = './'+item
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    
    dataset.to_csv(outdir+'/'+item+'.csv',index=False, sep= '\t')

    # data split into train, test, validation
    ## splite ratio: .6:.2:.2 (0-0.6; 0.6-0.8; 0.8-1)
    x,y,z = np.split(dataset.sample(frac=1, random_state=42), 
                       [int(.6*len(dataset)), int(.8*len(dataset))])
    splits = dict(zip(['train', 'test', 'validation'],[x,y,z]))
    for split in splits:
        splits[split].to_csv(outdir+'/'+split+'.csv', index=False,sep='\t')
    
    datasets.append(dataset)

/var/folders/k1/pt43vy6x3vl_nb58fb7wn9jh0000gn/T/ipykernel_31535/1987353984.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[item]= dataset[item].str.strip()


<class 'pandas.core.frame.DataFrame'>
Index: 149910 entries, 0 to 227826
Data columns (total 14 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   finding                           149910 non-null  object
 1   label_Atelectasis                 149910 non-null  int64 
 2   label_Cardiomegaly                149910 non-null  int64 
 3   label_Consolidation               149910 non-null  int64 
 4   label_Edema                       149910 non-null  int64 
 5   label_Enlarged Cardiomediastinum  149910 non-null  int64 
 6   label_Fracture                    149910 non-null  int64 
 7   label_Lung Lesion                 149910 non-null  int64 
 8   label_Lung Opacity                149910 non-null  int64 
 9   label_No Finding                  149910 non-null  int64 
 10  label_Pleural Effusion            149910 non-null  int64 
 11  label_Pleural Other               149910 non-null  int64 
 12  label_P

/var/folders/k1/pt43vy6x3vl_nb58fb7wn9jh0000gn/T/ipykernel_31535/1987353984.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[item]= dataset[item].str.strip()


<class 'pandas.core.frame.DataFrame'>
Index: 165619 entries, 0 to 227826
Data columns (total 14 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   impression                        165619 non-null  object
 1   label_Atelectasis                 165619 non-null  int64 
 2   label_Cardiomegaly                165619 non-null  int64 
 3   label_Consolidation               165619 non-null  int64 
 4   label_Edema                       165619 non-null  int64 
 5   label_Enlarged Cardiomediastinum  165619 non-null  int64 
 6   label_Fracture                    165619 non-null  int64 
 7   label_Lung Lesion                 165619 non-null  int64 
 8   label_Lung Opacity                165619 non-null  int64 
 9   label_No Finding                  165619 non-null  int64 
 10  label_Pleural Effusion            165619 non-null  int64 
 11  label_Pleural Other               165619 non-null  int64 
 12  label_P